In [1]:
%matplotlib inline
# Load all necessary packages
import sys
import numpy as np
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

2022-11-09 20:49:51.971505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 20:49:52.074932: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 20:49:52.486858: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-09 20:49:52.486910: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
def get_plain_and_debaised_model():
    ## trains on whole dataset/ no train-test split in this function
    dataset_orig= load_preproc_data_adult()
    dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]

    sess_plain = tf.Session()
    plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess_plain)

    plain_model.fit(dataset_orig_train)

    tf.reset_default_graph()
    sess_debiased = tf.Session()
    debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess_debiased)

    debiased_model.fit(dataset_orig_train)

    return plain_model, debiased_model


plain_model, debiased_model = get_plain_and_debaised_model()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2022-11-09 20:49:53.224910: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-09 20:49:53.246775: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-11-09 20:49:53.246792: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-11-09 20:49:53.247157: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

epoch 0; iter: 0; batch classifier loss: 0.663739
epoch 0; iter: 200; batch classifier loss: 0.421104
epoch 1; iter: 0; batch classifier loss: 0.449258
epoch 1; iter: 200; batch classifier loss: 0.456448
epoch 2; iter: 0; batch classifier loss: 0.495314
epoch 2; iter: 200; batch classifier loss: 0.419341
epoch 3; iter: 0; batch classifier loss: 0.434218
epoch 3; iter: 200; batch classifier loss: 0.391853
epoch 4; iter: 0; batch classifier loss: 0.497861
epoch 4; iter: 200; batch classifier loss: 0.408778
epoch 5; iter: 0; batch classifier loss: 0.401274
epoch 5; iter: 200; batch classifier loss: 0.423250
epoch 6; iter: 0; batch classifier loss: 0.478650
epoch 6; iter: 200; batch classifier loss: 0.405312
epoch 7; iter: 0; batch classifier loss: 0.404838
epoch 7; iter: 200; batch classifier loss: 0.386942
epoch 8; iter: 0; batch classifier loss: 0.392513
epoch 8; iter: 200; batch classifier loss: 0.460079
epoch 9; iter: 0; batch classifier loss: 0.443524
epoch 9; iter: 200; batch classi

In [3]:
def create_single_entry_adult(race, sex, age, education_years):
    arr = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

    arr[0][0] = race
    arr[0][1] = sex
    if age <= 19:
        arr[0][2] = 1.
    elif age <= 29:
        arr[0][3] = 1.
    elif age <= 39:
        arr[0][4] = 1.
    elif age <= 49:
        arr[0][5] = 1.
    elif age <= 59:
        arr[0][6] = 1.
    elif age <= 69:
        arr[0][7] = 1.
    else:
        arr[0][8] = 1.

    if education_years < 6:
        arr[0][16] = 1.
    elif education_years == 6:
        arr[0][9] = 1.
    elif education_years == 7:
        arr[0][10] = 1.
    elif education_years == 8:
        arr[0][11] = 1.
    elif education_years == 9:
        arr[0][12] = 1.
    elif education_years == 10:
        arr[0][13] = 1.
    elif education_years == 11:
        arr[0][14] = 1.
    elif education_years == 12:
        arr[0][15] = 1.
    else:
        arr[0][17] = 1.
    
    dataset_replaced_data = load_preproc_data_adult()
    dataset_replaced_data.features = arr
    dataset_replaced_data.age = age
    dataset_replaced_data.edu = education_years
    return dataset_replaced_data

# IDK what the race and sex is - that is, idk what sex = 1 means or race = 0
# not sure where to find that
example_input = create_single_entry_adult(1, 0, 65, 13)
# we can make a script that prompts a user to input sex, race, age, and education and input to this^^^

In [4]:
def predict_single_income(model, user_input):
    pred = model.predict(user_input).labels[0][0]
    
    race_print = f"(race={user_input.features[0][0]})" # idk which value is which race so this is the placeholder 
    sex_print = f"(sex={user_input.features[0][1]})" # idk which value is which sex so this is the placeholder 

    if pred == 1.0:
        print(f"The model predicts that a {user_input.age} year old {race_print} {sex_print} with {user_input.edu} years of education DOES have an income greater than 50k.")
    elif pred == 0.0:
        print(f"The model predicts that a {user_input.age} year old {race_print} {sex_print} with {user_input.edu} years of education DOES NOT have an income greater than 50k.")

predict_single_income(plain_model, example_input)

The model predicts that a 65 year old (race=1.0) (sex=0.0) with 13 years of education DOES NOT have an income greater than 50k.


In [5]:
input1 = create_single_entry_adult(1, 1, 65, 13)
input2 = create_single_entry_adult(1, 0, 65, 13)

predict_single_income(plain_model, input1)
predict_single_income(plain_model, input2)

The model predicts that a 65 year old (race=1.0) (sex=1.0) with 13 years of education DOES have an income greater than 50k.
The model predicts that a 65 year old (race=1.0) (sex=0.0) with 13 years of education DOES NOT have an income greater than 50k.


In [6]:
input1 = create_single_entry_adult(1, 1, 65, 11)
input2 = create_single_entry_adult(1, 0, 65, 11)

predict_single_income(debiased_model, input1)
predict_single_income(debiased_model, input2)

The model predicts that a 65 year old (race=1.0) (sex=1.0) with 11 years of education DOES NOT have an income greater than 50k.
The model predicts that a 65 year old (race=1.0) (sex=0.0) with 11 years of education DOES NOT have an income greater than 50k.
